## Notebook objectives

Classify Clusters in a Business sense and Exctract strategies to make more money out of them.


## Agenda

[Business Context](#Business-Context)<br> 
&emsp;[Business Questions](#Business-Context)<br>
&emsp;[Hypothesis](#Business-Context)<br>

[Imports](#Imports)<br>
&emsp;[Helper Functions and Classes](#Helper-Functions-and-Classes)<br>


[Settings](#Settings)<br>

[Loading Data](#Loading-Data)<br>
&emsp;[df_orders_abt](#df_orders_abt)<br>
&emsp;[df_clusters](#df_clusters)<br>

[Analytical Base Table](#Analytical-Base-Table)<br>
&emsp;[df_customer](#df_customer)<br>

[Analysis](#Analysis)<br>
[AgglomerativeClustering](#AgglomerativeClustering)<br>

[Next Steps](#Next-Steps)<br>

## Business Context

E-Mart wants to gain a better understanding of how It provides value to its customers. Based on customer understanding, the retailer will focus on increasing sales by creating messaging that resonates with target audience. In other words, **The ultimate goal of this project is being able to target our users differently given their current status**. 

Therefore, we need to identify customer personas and their value.

- **Who are our best customers?**
<br>

- **Where are they?**
<br>

- **Who has the potential to become valuable customers?**
<br>

- **Which of our customers are most likely to respond to engagement campaigns?**


## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



### Helper Functions and Classes

In [2]:
def get_snakecase_columns(df):
    """Sets column name to snake_case format
    
    df: pandas.DataFrame
    
    Return: map
    """
    snakecase = lambda x: str(x).lower().replace(' ', '_').replace('-', '_')
    return map(snakecase, df.columns)

def date(str_date):
    """Apply pandas.to_datetime to argument converting it to datetime.
        
    """
    return pd.to_datetime(str_date)

def find_column(df, col_name):
    """Checks if DataFrame contains a 'column name' and returns the matched columns
    
    df: pandas.DataFrame
    col_name: column name or part of column name to search for
    
    Return: DataFrame with column names that match the col_name searched
    """
    
    df_cols = pd.DataFrame(df.columns, columns=['col_name'])

    return df_cols[df_cols['col_name'].str.contains(col_name)].reset_index(drop=True)


def fig(x=15, y=5, set_as_global=False, reset_to_default=False):
    """ Adjust size of matplotlib figure

    x: figure width.
    y: figure height.
    set_as_global: bool.
        If True, then it sets "x" and "y" axis for all subsequent plots.
    reset_to_default: bool.
        If True, then it resets the global figure size back to default.
    """
    if set_as_global:
        plt.rcParams["figure.figsize"] = (x, y)
    elif reset_to_default:
        plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"]
    else:
        plt.figure(figsize=(x,y))
        
        
def count_plot(data, cluster_col="cluster", palette='crest_r', title='countplot'):
    plt.box(False)
    ax = sns.countplot(y=cluster_col, data=data, palette=palette)
    plt.title(title)
    plt.tick_params(axis='both', length=0)
    plt.xlabel(None)
    for p in ax.patches:
        ax.annotate('{:.0f}'.format(p.get_width()), (p.get_width()+5, p.get_y()+.5))
        

    
def cluster_stats(df, column, cluster='cluster', quantile_1=.8, quantile_2=.9):    
    stats = df.groupby(cluster)[[column]].describe()[column]
    quantiles = pd.concat([df.groupby(cluster)[[column]].quantile(quantile_1), df.groupby(cluster)[[column]].quantile(quantile_2)], axis=1)
    quantile_1_name, quantile_2_name = f'{round(quantile_1*100)}%', f'{round(quantile_2*100)}%'
    quantiles.columns = [quantile_1_name, quantile_2_name]
    stats = pd.concat([stats, quantiles], axis=1)
    return stats[['mean', 'std', 'min', '25%', '50%', '75%', quantile_1_name, quantile_2_name, 'max']]

    

## Settings

In [3]:
pd.set_option('display.max_columns', 500)

In [4]:
%matplotlib inline

## Loading Data

### df_orders_abt

In [5]:
df_orders_abt = pd.read_csv("../../data/growth_analysis/orders_abt.csv")

### df_clusters

In [6]:
df_clusters = pd.read_csv("../../data/customer_segmentation/agglomerative_clustering.csv")

### df_rfv

In [7]:
df_rfv = pd.read_csv("../../data/customer_segmentation/customer_rfv.csv")

## Analytical Base Table

## df_customer

In [8]:
df_customers = (df_orders_abt
                   .groupby("customer_id")
                   .agg(
                       orders=('order_id', 'nunique'),
                       total_sales=('sales', 'sum'),
                       avg_sales=('sales', 'mean'),
                       total_profit=('profit', 'sum'),
                       avg_profit=('profit', 'mean'),
                       total_products=('n_products', 'sum'),
                       avg_products=('n_products', 'mean'),
                       avg_shipping_cost=('shipping_cost', 'mean'),
                       avg_discount=('avg_discount', 'mean'),
                       profitability=('profitable', 'mean'),
                       delivery_on_time_rate=('delivery_on_time', 'mean'),
                       avg_days_to_ship=('days_to_ship', 'mean'),
                       avg_delivery_time=('delivery_time', 'mean'),
                   )
)
df_customers = (df_customers.merge(df_clusters, how='left', on='customer_id')
                            .merge(df_rfv, how='left', on='customer_id')
               )
df_customers.head()

,customer_id,orders,total_sales,avg_sales,total_profit,avg_profit,total_products,avg_products,avg_shipping_cost,avg_discount,profitability,delivery_on_time_rate,avg_days_to_ship,avg_delivery_time,cluster,rfv_class
0,AA-10315,19,35459.217783,1866.274620,447.69050,23.562658,145,7.631579,65.060526,0.097368,0.789474,0.736842,15.411765,21.941176,0,Potential Loyalist
1,AA-10375,23,11508.993783,500.391034,677.47740,29.455539,139,6.043478,39.300870,0.184420,0.695652,0.869565,15.772727,20.181818,0,Loyal
2,AA-10480,20,262423.012928,13121.150646,1516.47518,75.823759,150,7.500000,81.683500,0.113025,0.750000,0.950000,13.700000,17.750000,2,Need Attention
3,AA-10645,36,369585.731434,10266.270318,3051.43900,84.762194,267,7.416667,48.674167,0.122875,0.861111,0.972222,12.083333,18.085714,2,Loyal
4,AA-315,7,2216.891129,316.698733,535.56600,76.509429,20,2.857143,30.828571,0.257143,0.571429,1.000000,13.285714,17.714286,0,Potential Loyalist


## Analysis

### AgglomerativeClustering

Initial Cluster Segment Analysis

    Cluster 0 (Promising):
        High Number of Orders
    
    Cluster 1 (Risky):
        Negative Profit
        High Discounts
        Lower Shipping Cost
        
    Cluster 2 (Loyal):
        High Average Ticket
        High Number of Orders
        High Number of Products per Order
        High Shipping Cost
        Profitable Sales
        
        
    Cluster 3 (Champions):
        Lower Discounts
        Only Profitable Sales

**Final Group Characteristics**


        
        Loyal:
            295 Customers with
                78.71% of Total Revenue
                33.84% of Total Profit
                30.10% of Number of Orders
                Avg Profit: $ 68.72
                Avg Discount: 13.10% (3.65% up to 21%)
        
        Potential Loyalist:
            1026 Customers with
                20.89% of Total Revenue
                59.63% of Total Profit
                64.29% of Number of Orders
                Avg Profit: $ 52.22
                Avg Discount: 14.68% (0% up to 30%)
                
        Promising:
            123 Customers with
                0.14% of Total Revenue
                3.14% of Total Profit
                2.30% of Number of Orders
                Avg Profit: $ 82.77
                Avg Discount: 0.02% (0% up to 2%)
                
        Risky:
             146 Customers with
                0.24% of Total Revenue
                -3.36% of Total Profit
                3.30% of Number of Orders
                Avg Profit: $ -65.83
                Avg Discount: 36.24% (20% up to 70%)

In [9]:
df_aux = df_customers.copy()
df_aux['cluster'] = df_aux['cluster'].map(
        {0:'Potential Loyalist',
         1:'Risky',
         2:'Loyal',
         3:'Promising',}
    )
df_aux['cluster'] = pd.Categorical(df_aux['cluster'], ["Risky", "Promising", "Potential Loyalist", "Loyal"])

df_stats = (df_aux.drop(columns=['customer_id', 'rfv_class'])
                 .groupby(['cluster']).mean().sort_values('cluster', ascending=False)
           )
df_stats

,orders,total_sales,avg_sales,total_profit,avg_profit,total_products,avg_products,avg_shipping_cost,avg_discount,profitability,delivery_on_time_rate,avg_days_to_ship,avg_delivery_time
cluster,,,,,,,,,,,,,
Loyal,26.277966,367223.535513,14157.325878,1805.180544,68.728995,200.413559,7.630598,58.266722,0.131018,0.762206,0.907645,13.618051,18.752693
Potential Loyalist,16.137427,28022.559981,1206.798380,914.500481,52.222134,109.460039,5.977982,47.165310,0.146856,0.748279,0.909214,13.349012,18.434096
Promising,4.821138,1581.068450,319.265810,402.859268,82.773563,20.162602,4.260101,34.403870,0.000230,1.000000,0.887276,13.340112,18.548529
Risky,5.828767,2333.089673,288.314936,-362.329816,-65.833372,30.164384,4.830547,30.621937,0.362447,0.419888,0.920787,13.797365,18.884448


In [10]:
df_aux['cluster'].value_counts()

cluster
Potential Loyalist    1026
Loyal                  295
Risky                  146
Promising              123
Name: count, dtype: int64

In [11]:
cols = ['total_sales', 'total_profit', 'orders', 'total_products']
df_stats = df_aux.groupby(['cluster'])[cols].sum().T
df_stats['total'] = df_stats.abs().sum(axis=1)
df_stats.div(df_stats['total'], axis=0) *100

cluster,Risky,Promising,Potential Loyalist,Loyal,total
total_sales,0.247521,0.141313,20.892118,78.719048,100.0
total_profit,-3.362460,3.149623,59.639152,33.848765,100.0
orders,3.304469,2.302644,64.291539,30.101347,100.0
total_products,2.469828,1.390821,62.982862,33.156490,100.0


In [12]:
cluster_stats(df_aux, 'avg_discount', quantile_2=.95).sort_values("cluster", ascending=False)

,mean,std,min,25%,50%,75%,80%,95%,max
cluster,,,,,,,,,
Loyal,0.131018,0.034556,0.036583,0.108218,0.130624,0.151225,0.157046,0.191834,0.218418
Potential Loyalist,0.146856,0.055624,0.000000,0.108333,0.144372,0.177206,0.190000,0.250000,0.304762
Promising,0.000230,0.001948,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000
Risky,0.362447,0.102513,0.000000,0.300000,0.337500,0.400000,0.416667,0.591667,0.700000


In [13]:
# Due to use All Timeline to Create Clusters
df_aux_rfv = (df_aux.groupby(["cluster", "rfv_class"])[['customer_id']].nunique().reset_index()
                    .merge(df_aux.groupby(["cluster"]).agg(n_customers=('customer_id','nunique')),
                           how='left', on='cluster')
)

df_aux_rfv['perc'] = df_aux_rfv['customer_id'] / df_aux_rfv['n_customers']
df_aux_rfv.sort_values(['cluster', 'perc'], ascending=False)

,cluster,rfv_class,customer_id,n_customers,perc
35,Loyal,Loyal,234,295,0.793220
38,Loyal,Potential Loyalist,29,295,0.098305
36,Loyal,Need Attention,20,295,0.067797
33,Loyal,Champion,8,295,0.027119
32,Loyal,At Risk,4,295,0.013559
30,Loyal,About to Lose,0,295,0.000000
31,Loyal,About to Sleep,0,295,0.000000
34,Loyal,Hibernating,0,295,0.000000
37,Loyal,New Customer,0,295,0.000000
39,Loyal,Promising,0,295,0.000000


## Next Steps

Take more variables in consideration when creating customer segments

Market Basket Analysis - What Products will maximize profit when cross-selling?

What Exatcly Causes Negative Profit? Specific city, category, product?